In [36]:
import json
import sys
import copy

CorpusFile=open('./corpus/zh_train_tagged.txt','r')
Corpus=[i.strip().split(' ') for i in CorpusFile.readlines()]
CorpusFile.close()

A={}
B={}

A['start']={}
A['start']['start']={}
A['end']={}

for line in Corpus:
    start=line[0].rsplit('/',1)
    end0=line[-2].rsplit('/',1)
    end1=line[-1].rsplit('/',1)
    A['start']['start'][start[1]]=A['start']['start'].get(start[1],0)+1
    A['end'][end0[1]]=A['end'].get(end0[1],{})
    A['end'][end0[1]][end1[1]]=A['end'][end0[1]].get(end1[1],0)+1
    B[end1[1]]=B.get(end1[1],{})
    B[end1[1]][end1[0]]=B[end1[1]].get(end1[0],0)+1
    cur_word=start
    next_word=start
    for i in range(len(line)-1):
        pre_word=cur_word
        cur_word=next_word
        next_word=line[i+1].rsplit('/',1)
        A[pre_word[1]]=A.get(cur_word[1],{})
        A[pre_word[1]][cur_word[1]]=A[pre_word[1]].get(cur_word[1],{})
        A[pre_word[1]][cur_word[1]][next_word[1]]=A[pre_word[1]][cur_word[1]].get(next_word[1],0)+1
    
        B[cur_word[1]]=B.get(cur_word[1],{})
        B[cur_word[1]][cur_word[0]]=B[cur_word[1]].get(cur_word[0],0)+1  

totalTrans=len(B)
totalWords=0
A0=copy.deepcopy(A)

for k in B:
    totalWords=totalWords+len(B.get(k))
    A['start']['start'][k]=A['start']['start'].get(k,0)
    
        
for k in A0:
    for l in B:
        if l in A0[k]:
            items=A0[k].get(l)
            lenOfItems=len(items)
            total=sum(items.values())
            pTrans=1/(total+lenOfItems)
            for trans in B.keys():
                if trans in items:
                    A[k][l][trans]=A[k][l][trans]/total-pTrans
                else:
                    A[k][l][trans]=lenOfItems*pTrans/(totalTrans-lenOfItems)
        else:
            A[k][l]={x:1/totalTrans for x in B.keys()}
for k in B:
    if not k in A0:
        for l in B:
            A[k][l]={x:1/totalTrans for x in B.keys()}
for k in B:
    items=B.get(k)
    total=sum(items.values())
    pEm=1/(total+len(items))
    for em in items:
        B[k][em]=B[k][em]/total-pEm
    B[k]['other']=len(items)*pEm/(totalWords-len(items))
        
Model=[A,B]
ModelFile=open('./hmmmodel.txt','w')
ModelFile.write(json.dumps(Model))
ModelFile.close()

In [37]:
import json
import sys

InputFile=open('./corpus/zh_dev_raw.txt','r')
Input=[i.strip().split(' ') for i in InputFile.readlines()]


ModelFile=open('./hmmmodel.txt')
Model=json.loads(ModelFile.read())
ModelFile.close()

A=Model[0]
B=Model[1]
BK=B.keys()

output=''
for sentence in Input:
    viterbi={}
    backpointer={}
    viterbi[0]={}
    backpointer[0]={}
    #Initialization
    for state in BK:
        emB=1/len(BK)
        if not sentence[0] in B[state]:
            emB=B[state]['other']
            if sentence[0][-1].isdigit() or sentence[0][0].isdigit() or sentence[0][-1]=='%':
                if(state=='CD'):
                    emB=1
                else:
                    emB=0
            if sentence[0][0:4]==('http' or 'www') or sentence[0][-3:]=='com':
                if(state=='ADD'):
                    emB=1
                else:
                    emB=0
        else:
            emB=B[state][sentence[0]]
        viterbi[0][state]=A['start']['start'][state]*emB
        backpointer[0][state]='start'
    for t in range(len(sentence)-1):
        viterbi[t+1]={}
        backpointer[t+1]={}
        for state in BK:
            maxK=''
            maxV=0
            emB=0
            if not sentence[t+1] in B[state]:
                emB=B[state]['other']
                if sentence[t+1][-1].isdigit() or sentence[t+1][0].isdigit() or sentence[t+1][-1]=='%':
                    if(state=='CD'):
                        emB=1
                    else:
                        emB=0
                if sentence[t+1][0:4]==('http' or 'www') or sentence[t+1][-3:]=='com':
                    if(state=='ADD'):
                        emB=1
                    else:
                        emB=0
            else:
                emB=B[state][sentence[t+1]]
                
            for s in BK:
                for u in BK:
                    V=viterbi[t][s]*A[s][u][state]*emB
                    if V>=maxV:
                        maxV=V
                        maxK=s
            viterbi[t+1][state]=maxV
            backpointer[t+1][state]=maxK
    maxK=''
    maxV=0
    for s in BK:
        for u in BK:
            V=viterbi[len(sentence)-1][s]*A['end'][s][u]
            if V>=maxV:
                maxV=V
                maxK=s
    viterbi['end']=maxV
    backpointer['end']=maxK
    tag=backpointer['end']
    for i in range(len(sentence)):
        sentence[-i-1]=sentence[-i-1]+'/'+tag+' '
        tag=backpointer[len(sentence)-i-1][tag]
    output=output+''.join(sentence).strip()+'\n'
OutputFile=open('./hmmoutput.txt','w', encoding='utf8')
OutputFile.write(output)
OutputFile.close()

In [39]:
OutputFile=open('./hmmoutput.txt','r')
StandardFile=open('./corpus/zh_dev_tagged.txt','r')
X=[i.strip().split(' ') for i in OutputFile.readlines()]
Y=[i.strip().split(' ') for i in StandardFile.readlines()]
OutputFile.close()
StandardFile.close()
Stat={}
total=0
incorrect=0
for i in range(len(X)):
    for j in range(len(X[i])):
        total=total+1
        if not X[i][j]==Y[i][j]:
            incorrect=incorrect+1
            x=X[i][j].rsplit('/',1)[1]
            y=Y[i][j].rsplit('/',1)[1]
            if(x=='VV' and y=='JJ'):
                print('F: '+X[i][j]+' T: '+Y[i][j]+'\n')
            Stat[y]=Stat.get(y,{})
            Stat[y][x]=Stat[y].get(x,0)+1
print(1-incorrect/total)

F: 相反/VV T: 相反/JJ

F: 無關/VV T: 無關/JJ

F: 相反/VV T: 相反/JJ

0.7084419173971412


In [35]:
A['start']['start']['.']

KeyError: '.'